# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#Random Forest
from imblearn.ensemble import BalancedRandomForestClassifier

#Easy Ensemble Classifier
from imblearn.ensemble import EasyEnsembleClassifier 

In [7]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [8]:
Counter(df.loan_status)

Counter({'low_risk': 68470, 'high_risk': 347})

## Read the CSV and Perform Basic Data Cleaning

In [9]:
# Load the data
df = pd.read_csv('LoanStats_2019Q1.csv')

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [10]:
# Get info on the number of rows so that we can split the data appropriately.
df.shape

# There are 68817 rows of data and 86 columns. 
# Following the 80-20 rule, 55,054 rows should be for training and the remaining data should be used for testing

(68817, 86)

In [11]:
# We must identify the column that is the Y variable. Not all columns are visible in the dataframe.
df.columns

Index(['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_si

## Split the Data into Training and Testing

In [12]:
# Create our features
X = df.drop(columns='loan_status')
X = pd.get_dummies(X)

# Create our target
Y = df['loan_status']

# Use the label encoder to create a binary class
#This transformer should be used to encode target values, *i.e.* `y`, and not the input `X`.

encoder = LabelEncoder()
encoder.fit(Y)
Y_code = encoder.transform(Y)
Y = Y_code

In [13]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [14]:
# Check the balance of our target values
Counter(Y)

Counter({1: 68470, 0: 347})

The data appears to be quite imbalanced

In [15]:
# Split the X and y into X_train, X_test, y_train, y_test
#Following the 80-20 rule, we want 80% of our data for training the algorithm.
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [16]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [17]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler.fit(X_train)

StandardScaler()

In [18]:
# Scale the training and testing data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

Definition: A balanced random forest randomly under-samples each boostrap sample to balance it.

In [19]:
# Resample the training data with the BalancedRandomForestClassifier
#Using a max depth of 5 so the trees don't go too far down
#n_estimators set at 1000 so we iterate and create a lot of trees to use
clf_model = BalancedRandomForestClassifier(max_depth=5, random_state=1, n_estimators=1000)
clf_model.fit(X_train, y_train)

BalancedRandomForestClassifier(max_depth=5, n_estimators=1000, random_state=1)

In [20]:
# Calculated the balanced accuracy score
clf_y_pred = clf_model.predict(X_test)
balanced_accuracy_score(y_test, clf_y_pred)

0.748484666293595

In [21]:
# Display the confusion matrix
confusion_matrix(y_test , clf_y_pred)

array([[   44,    35],
       [  821, 12864]], dtype=int64)

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, clf_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.05      0.56      0.94      0.09      0.72      0.50        79
          1       1.00      0.94      0.56      0.97      0.72      0.54     13685

avg / total       0.99      0.94      0.56      0.96      0.72      0.54     13764



In [32]:
# List the features sorted in descending order by feature importance

sorted(zip(clf_model.feature_importances_, X.columns), reverse=True)

[(0.11232220342628703, 'total_rec_prncp'),
 (0.10001352298913953, 'total_pymnt'),
 (0.0981275256686964, 'total_rec_int'),
 (0.09719607077282885, 'last_pymnt_amnt'),
 (0.09019945302039808, 'total_pymnt_inv'),
 (0.03497371388449912, 'int_rate'),
 (0.02474292217354763, 'issue_d_Jan-2019'),
 (0.01752727982435498, 'issue_d_Mar-2019'),
 (0.016370338416087715, 'mths_since_recent_inq'),
 (0.013667485597515712, 'dti'),
 (0.013024791057471525, 'installment'),
 (0.012533639249974374, 'max_bal_bc'),
 (0.011032342790675293, 'revol_bal'),
 (0.010876308828400485, 'total_rec_late_fee'),
 (0.01078530407562567, 'out_prncp'),
 (0.010525122420341084, 'bc_util'),
 (0.010488976115201365, 'out_prncp_inv'),
 (0.010379917377018754, 'tot_hi_cred_lim'),
 (0.009873349771889759, 'total_bc_limit'),
 (0.009858491673393761, 'mths_since_rcnt_il'),
 (0.009833842553168588, 'total_bal_il'),
 (0.009833286243294947, 'tot_cur_bal'),
 (0.009688916964917816, 'annual_inc'),
 (0.009686584383504284, 'inq_last_12m'),
 (0.00901676

### Easy Ensemble Classifier

In [35]:
# Train the Classifier
eec = EasyEnsembleClassifier(random_state=1)
eec_model = eec.fit(X_train, y_train)

In [36]:
# Calculated the balanced accuracy score
eec_y_pred = eec_model.predict(X_test)
balanced_accuracy_score(y_test, eec_y_pred)

0.9164395092103985

In [37]:
# Display the confusion matrix
confusion_matrix(y_test , eec_y_pred)

array([[   70,     9],
       [  728, 12957]], dtype=int64)

In [38]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, eec_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.09      0.89      0.95      0.16      0.92      0.83        79
          1       1.00      0.95      0.89      0.97      0.92      0.84     13685

avg / total       0.99      0.95      0.89      0.97      0.92      0.84     13764



### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble Classifeir model was much better with a balanced accuracy score of 0.9164395092103985
2. Which model had the best recall score?

    The Easy Ensemble Classifeir model was better with a recall score of 0.95. The recall specific to high-risk loans was also more than twice as accurate as the random forest.

3. Which model had the best geometric mean score?

    The Easy Ensemble Classifeir model had a better with a geometric mena score of 0.92 which is 20 points higher than the random forest.

4. What are the top three features?

 (0.11232220342628703, 'total_rec_prncp'),
 (0.10001352298913953, 'total_pymnt'),
 (0.0981275256686964, 'total_rec_int')